## Yearly Comparison of Job Offer Submission Counts

In [1]:
import itertools
import datetime as dt
import psycopg2
import pandas as pd
import plotly
import chart_studio.plotly as py
import plotly.graph_objs as go

from nbstyler import DATA_STYLE as DS

plotly.offline.init_notebook_mode(connected=True)

%matplotlib notebook
%matplotlib inline

### Objectives

A line chart comparing the total number of job offer submissions weekly for each year since the project started. The x axis will start from 0 and continue up to week 52, and over the y axis the lines for each respective year will be shown.


### Data Preparation

Get the data and prepare the `Pandas.DataFrame` object.

In [2]:
with psycopg2.connect('dbname=jobsbg') as conn:
    weekly_df = pd.read_sql_query('SELECT * FROM v_all_offers_count_weekly', conn, index_col='week_ts')

weekly_df.index = pd.to_datetime(weekly_df.index)
weekly_df.head(5)

,subm_count
week_ts,
2017-09-25,5287
2017-10-02,10281
2017-10-09,9501
2017-10-16,9672
2017-10-23,9303


Adjust week numbers and prepare a dictionary holding the data for each year. 

To adjust the week numbers for each year I used https://www.epochconverter.com.

In [3]:
yearly_dfs = {
    '2017': weekly_df[(weekly_df.index >= '2017-01-01') & (weekly_df.index <= '2017-12-31')],
    '2018': weekly_df[(weekly_df.index >= '2018-01-01') & (weekly_df.index <= '2018-12-30')],
    '2019': weekly_df[(weekly_df.index >= '2018-12-31') & (weekly_df.index <= '2019-12-29')],
    '2020': weekly_df[(weekly_df.index >= '2019-12-30') & (weekly_df.index <= '2020-03-28')],
}

Preparing the line objects

In [4]:
line_2018 = go.Scatter(
    x=[n for n in range(1, 53)],
    y=[value for value in yearly_dfs['2018'].subm_count],
    name='2018',
    hovertemplate='w%{x}: %{y:,}',
    mode='lines',
    showlegend=True,
    line=dict(
        color=DS['colorramp']['acc2'][3],
    ),
    visible=True,
)

line_2019 = go.Scatter(
    x=[n for n in range(1, 53)],
    y=[value for value in yearly_dfs['2019'].subm_count],
    name='2019',
    hovertemplate='w%{x}: %{y:,}',
    mode='lines',
    showlegend=True,
    line=dict(
        color=DS['colorramp']['acc1'][3],
    ),
    visible=True,
)

line_2020 = go.Scatter(
    x=[n for n in range(1, 53)],
    y=[value for value in yearly_dfs['2020'].subm_count],
    name='2020',
    mode='lines',
    hovertemplate='w%{x}: %{y:,}',
    showlegend=True,
    line=dict(
        color=DS['colorramp']['acc1'][6],
        width=3,
    ),
    visible=True,
)


data = [line_2018, line_2019, line_2020]

In [5]:
quarter_anno = go.Scatter(
    x=[i for i in [1, 13, 26, 39]],
    y=[500, 500, 500, 500],
    text=['Q1', 'Q2', 'Q3', 'Q4'],
    mode='text',
    showlegend=False,
    textfont=DS['chart_fonts']['anno']
)

In [6]:
data = [line_2018, line_2019, line_2020, quarter_anno]

Preparing rectangle shapes that will show the quarters' span.

In [7]:
# Highlight quarters with Shape-s
shapes=[
        dict(
            type='rect',
            name='Q1',
            xref='x',
            yref='y',
            x0=1,
            y0=0,
            x1=13,
            y1=15000,
            fillcolor=DS['colors']['bg1'],
            layer="below",
            line_width=0,
        ),dict(
            type='rect',
            name='Q2',
            xref='x',
            yref='y',
            x0=13,
            y0=0,
            x1=26,
            y1=15000,
            fillcolor=DS['colors']['bg2'],
            layer="below",
            line_width=0,
        ),dict(
            type='rect',
            name='Q3',
            xref='x',
            yref='y',
            x0=26,
            y0=0,
            x1=39,
            y1=15000,
            fillcolor=DS['colors']['bg1'],
            layer="below",
            line_width=0,
        ),dict(
            type='rect',
            name='Q4',
            xref='x',
            yref='y',
            x0=39,
            y0=0,
            x1=52,
            y1=15000,
            fillcolor=DS['colors']['bg2'],
            layer="below",
            line_width=0,
        ),
]

Preparing the layout

In [8]:
layout = go.Layout(
    paper_bgcolor=DS['colors']['bg1'],
    plot_bgcolor=DS['colors']['bg1'],
    title='Yearly Comparison of Job Offer Submission Counts',
    titlefont=DS['chart_fonts']['title'],
    font=DS['chart_fonts']['text'],
    legend=dict(
        orientation='v',
        bgcolor=DS['colors']['bg2'],
        bordercolor=DS['colors']['bg3'],
        borderwidth=1,
    ),
    autosize=True,
    showlegend=True,
    hidesources=True,
    dragmode='zoom',
    hovermode='closest',
    xaxis_title='Week',
    yaxis_title='Number of Submissions',
    xaxis=dict(
        range=(0,52),
        ticks='outside',
        tickmode='auto',
        nticks=52,
        zerolinecolor=DS['colors']['bg3'],
        gridcolor=DS['colors']['bg3'],
    ),
    yaxis=dict(
        range=(0,14000),
        fixedrange=True,
        ticks='outside',
        tickwidth=1,
        zerolinecolor=DS['colors']['bg3'],
        gridcolor=DS['colors']['bg3'],
    ),
    shapes=shapes
)

In [9]:
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='all_offers_weekly_yoy_submissions_line.html')

In [10]:
# Uncomment the line below to export an HTML version of the chart.
plotly.offline.plot(fig, filename='all_offers_weekly_yoy_submissions_line.html', show_link=False)

'all_offers_weekly_yoy_submissions_line.html'

In [11]:
from IPython.core.display import HTML
with open('../resources/styles/datum.css', 'r') as f:
    style = f.read()
HTML(style)